## Simulate the runout of a pile of debris that suddenly collapses

### import necessary packages and components

In [ ]:
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from matplotlib import cm
import numpy as np

from landlab import RasterModelGrid
from landlab.components import FlowAccumulator, FlowDirectorMFD
from landlab import imshowhs_grid
from landlab.components.mass_wasting_router import MassWastingRunout

### define the model domain, geometry of the pile and MassWastingRunout parameters Sc, qsi and k

In [ ]:
# model domain
r = 51 # rows in model domain
c = r # columns
dxdy = 5 # cell size
hs = 1 # soil thickness
Dp = 0.2 # maximum particle diameter in soil

# define the pile
ls_h = 5 # height of pile
w = 5 # width of pile

# MassWastingRunout parameters
Sc = [0.03]  # critical slope
qsi = 0.01 # threshold flux
k = 0.02 # scour coefficient

### create model grid and add necessary grid fields

In [ ]:
mg = RasterModelGrid((r,c),dxdy)

dem = mg.add_field('topographic__elevation',
                    np.ones(r*c)*1,
                    at='node')

mg.at_node['node_id'] = np.hstack(mg.nodes)
# domain for plots
xmin = mg.node_x.min(); xmax = mg.node_x.max(); ymin = mg.node_y.min(); ymax = mg.node_y.max()
# set boundary conditions, add flow direction
mg.set_closed_boundaries_at_grid_edges(True, True, True, True) #close all boundaries
# flow directions
fa = FlowAccumulator(mg, 
                      'topographic__elevation',
                      flow_director='FlowDirectorD8')
fa.run_one_step()
# soil thickness
thickness = np.ones(mg.number_of_nodes)*hs
mg.add_field('node', 'soil__thickness',thickness)
# set particle diameter
mg.at_node['particle__diameter'] = np.ones(len(mg.node_x))*Dp
# copy of initial topography
DEMi = mg.at_node['topographic__elevation'].copy()

### run flow director to add slope and receiving node fields

In [ ]:
mg.delete_field(loc = 'node', name = 'flow__sink_flag')
mg.delete_field(loc = 'node', name = 'flow__link_to_receiver_node')
mg.delete_field(loc = 'node', name = 'flow__receiver_node')
mg.delete_field(loc = 'node', name = 'topographic__steepest_slope')
fd = FlowDirectorMFD(mg, diagonals=True,
                      partition_method = 'square_root_of_slope')
fd.run_one_step()

### create the pile

In [ ]:
# find central point in domain
x = mg.node_x.max()/2
y = mg.node_y.max()/2

# find all nodes with radius of central point
dn = ((mg.node_x-x)**2+(mg.node_y-y)**2)**0.5
pile_nodes = np.hstack(mg.nodes)[dn<w*mg.dx]

# set those nodes as the landslide (they're all given the same mass__wasting_id number)
mg.at_node['mass__wasting_id'] = np.zeros(mg.number_of_nodes).astype(int)
mg.at_node['mass__wasting_id'][pile_nodes] = 1

# set thickness of landslide (in this case the pile)
mg.at_node['soil__thickness'][pile_nodes] = ls_h
mg.at_node['topographic__elevation'][pile_nodes] =   mg.at_node['topographic__elevation'][pile_nodes]+(ls_h-hs)

### take a look at the pile

In [ ]:
# borrow the plotting routine from the flow-director notebookks to make a 3d plot of our surface.
# import plotting tools
def surf_plot(mg,
              surface='topographic__elevation',
              title='Surface plot of topography'):

    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    # Plot the surface.
    Z = mg.at_node[surface].reshape(mg.shape)
    print('Zmin{}, Zmax{}'.format(Z.min(), Z.max()))
    color = cm.gray_r((Z - Z.min())*2 / (Z.max() - Z.min()))
    surf = ax.plot_surface(mg.x_of_node.reshape(mg.shape),
                           mg.y_of_node.reshape(mg.shape),
                           Z,
                           rstride=1,
                           cstride=1,
                           facecolors=color,
                           linewidth=0.,
                           antialiased=False,
                          alpha = 0.67)
    ax.view_init(elev=20, azim=-75)
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Elevation')
    plt.title(title)
    ax.set_box_aspect([1, 1, 0.2])
    plt.show()
    
# 3d view
surf_plot(mg)
# 2d view
plt.figure(figsize = (7,7))
imshowhs_grid(mg,
            'topographic__elevation',
            drape1 = 'topographic__elevation',
            cmap = 'terrain',
            allow_colorbar=True,
            plot_type=  "Drape1",
            limits = (0,5),
            alpha = 0.75,
            altdeg=37., azdeg=210)

### that's a nice looking pile...now set up an instance of MassWastingRuntout to model its collapse and runout 

In [ ]:
mw_dict = {'critical slope':Sc, 'threshold flux':qsi,
            'scour coefficient':k, 'effective particle diameter':Dp}
MWRu = MassWastingRunout(mg, mw_dict, effective_qsi = False, save = True, grain_shear = False, settle_deposit = True)

### run the instance of MassWastingRunout

In [ ]:
start_time = time.time()
MWRu.run_one_step(run_id = 0)

### view runout extent

In [ ]:
# 3d view
surf_plot(mg)

#2d view
plt.figure(figsize = (7,7))
imshowhs_grid(mg,
            'topographic__elevation',
            drape1 = 'topographic__elevation',
            cmap = 'terrain',
            allow_colorbar=True,
            plot_type=  "Drape1",
            limits = (0,5),
            alpha = 0.75,
            altdeg=37., azdeg=210)

### view topographic change caused by the runout

In [ ]:
mg.at_node['dem_dif_o'] = mg.at_node['topographic__elevation']-mg.at_node['topographic__initial_elevation']

plt.figure(figsize = (7,7))
imshowhs_grid(mg,
            'dem_dif_o',
            drape1 = 'dem_dif_o',
            cmap = 'RdBu_r',
            allow_colorbar=True,
            plot_type=  "Drape1",
            limits = (-0.5,0.5),
            alpha = 0.75,
            altdeg=37., azdeg=210)


### view of the pile as it ranout

#### first in 3d view

In [ ]:
# plot how DEM changes
for i in np.arange(0,len(MWRu.mw_ids)):
    for c in MWRu.runout_evo_maps[i].keys():                  
        plt.figure('topography'+str(c)+str(i),figsize=(12, 12))
        mg.at_node['topographic__elevation'] = MWRu.runout_evo_maps[i][c]
        # imshow_grid_at_node(mg,'df_topo',cmap ='RdBu_r')  
        #LLT.plot_node_field_with_shaded_dem(mg,field = 'df_topo', fontsize = 10,cmap = 'RdBu_r', plot_name = 'topo + thick{},{}'.format(i,c) )
        field = "node_id"
        #plt.figure(figsize = (7,7))
        
        surf_plot(mg, 'topographic__elevation')
        
        #imshowhs_grid(mg,
        #            'topographic_elevation',
        #            drape1 = 'topographic_elevation',
        #            cmap = 'terrain',
        #            allow_colorbar=True,
        #            plot_type=  "Drape1",
        #            limits = (0,5),
        #            alpha = 0.75,
        #            altdeg=37., azdeg=210)
        
        
        # plot_values(mg,field,xmin,xmax,ymin,ymax,field_back= field_back, background = False)
        #plt.xticks(fontsize= 8 )
        #plt.yticks(fontsize= 8 )
        #plt.xlim([xmin*.8,xmax*1.2]); plt.ylim([ymin*.3,ymax])
        #plt.show()

        if c >50:
            break

#### now in 2d view

In [ ]:
# plot how DEM changes
for i in np.arange(0,len(MWRu.mw_ids)):
    for c in MWRu.runout_evo_maps[i].keys():                  
        plt.figure('topography'+str(c)+str(i),figsize=(12, 12))
        mg.at_node['topographic__elevation'] = MWRu.runout_evo_maps[i][c]
        field = "node_id"
        plt.figure(figsize = (7,7))
        
        imshowhs_grid(mg,
                    'topographic__elevation',
                    drape1 = 'topographic__elevation',
                    cmap = 'terrain',
                    allow_colorbar=True,
                    plot_type=  "Drape1",
                    limits = (0,5),
                    alpha = 0.75,
                    altdeg=37., azdeg=210)
        plt.xticks(fontsize= 8 )
        plt.yticks(fontsize= 8 )
        plt.xlim([xmin*.8,xmax*1.2]); plt.ylim([ymin*.3,ymax])
        plt.show()

        if c >50:
            break

#### and finally, in profile view

In [ ]:
pf = mg.nodes[int((r-1)/2),:]
x_ = mg.node_x[pf]
y = mg.at_node['topographic__initial_elevation'][pf]
for i in np.arange(0,len(MWRu.mw_ids)):
    for c in MWRu.runout_evo_maps[i].keys():                  
        etopo = MWRu.runout_evo_maps[i][c]#-mg.at_node['topographic__initial_elevation']
        topo = MWRu.topo_evo_maps[i][c]               
        y_ = topo[pf]
        _y_ = etopo[pf]
        plt.figure(figsize = (6,3))
        plt.plot(x_,y,'k--', alpha = 0.5, linewidth = 1,label = 'initial ground profile')
        plt.plot(x_,y_,'g-', alpha = 0.5, linewidth = 1, label = 'ground profile, next iteration')
        plt.plot(x_,_y_,'r-', alpha = 0.5, linewidth = 1, label = 'debriton surface')
        plt.ylim(0,ls_h*3); plt.ylabel(' elevation ')
        plt.xlim([0, max(mg.node_x)])
        plt.legend()
        plt.grid(alpha = 0.5)  
        plt.title('iteration '+str(c))
